In [13]:
import pandas as pd
import os
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
train=pd.read_csv('../../data/AES/train_ori.csv')
train = train.reset_index(drop=True)
train['fold'] = 0
skf = StratifiedKFold(n_splits=5, random_state=2025, shuffle=True)
for fold, (train_index, test_index) in enumerate(skf.split(train, train['score'].values)):
    train.loc[test_index, 'fold'] = fold
print(train['fold'].value_counts())
for fold in range(5):
    tra = train[train['fold'] != fold]
    dev = train[train['fold'] == fold]
    folder_path = f'../../data/AES/fold{fold}/'
    os.makedirs(folder_path, exist_ok=True) 
    tra.to_csv(os.path.join(folder_path, 'train.csv'), index=False)
    dev.to_csv(os.path.join(folder_path, 'val.csv'), index=False)

fold
0    3462
1    3462
3    3461
4    3461
2    3461
Name: count, dtype: int64


In [15]:
train.columns

Index(['essay_id', 'full_text', 'score', 'fold'], dtype='object')

In [30]:
import pandas as pd
from utils.message import *
from utils.prompt import *
import json
import os
input_dir='../../data/AES/fold0/val_Qwen2.5-72B-Instruct.csv'
type_name='train' if 'train' in input_dir else 'val'
df=pd.read_csv(input_dir)
fold_name=input_dir.split('/')[-2]
df['pred_gt']=df['pred_gt'].apply(lambda x:str(x))
df['score']=df['score'].apply(lambda x:str(x))
df=df[df['pred_gt']==df['score']]

In [31]:
humans = df['truncated_text'].tolist()
gpts = df['reason_gt'].tolist()

In [32]:
messages = create_sft_dataset(system_cot_infer,humans,gpts)
output_path = f"/mnt/afs1/panhaojun/project/LLaMA-Factory/data/aes_{type_name}_{fold_name}.json"
# os.makedirs(os.path.dirname(output_path), exist_ok=True)
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(messages, f, ensure_ascii=False, indent=2)
print(f"✅ message 已保存到: {output_path}")

✅ message 已保存到: /mnt/afs1/panhaojun/project/LLaMA-Factory/data/aes_val_fold0.json


In [34]:
dataset_name=output_path.split('/')[-1].replace('.json','')

In [35]:
dataset_name

'aes_val_fold0'

In [36]:
# 加载已有的 JSON 文件
    
dataset_info_path='/mnt/afs1/panhaojun/project/LLaMA-Factory/data/dataset_info.json'
if os.path.exists(dataset_info_path):
    with open(dataset_info_path, "r", encoding="utf-8") as f:
        dataset_info = json.load(f)

# 添加新的字段
dataset_info[dataset_name] = {
    "file_name": f"{dataset_name}.json",
    "formatting": "sharegpt",
    "columns": {
        "messages": "conversations",
        "system": "system",
    }
}

# 保存回原文件
with open(dataset_info_path, "w", encoding="utf-8") as f:
    json.dump(dataset_info, f, ensure_ascii=False, indent=2)

print(f"✅ dataset_info['{dataset_name}'] 已成功添加并保存到 {dataset_info_path}")


✅ dataset_info['aes_val_fold0'] 已成功添加并保存到 /mnt/afs1/panhaojun/project/LLaMA-Factory/data/dataset_info.json


In [29]:
dataset_name

'aes_train_fold0'

✅ message 已保存到: /mnt/afs1/panhaojun/project/LLaMA-Factory/data/aes_train_fold0.json
✅ dataset_info['aes_train_fold0'] 已成功添加并保存到 /mnt/afs1/panhaojun/project/LLaMA-Factory/data/dataset_info.json
✅ message 已保存到: /mnt/afs1/panhaojun/project/LLaMA-Factory/data/aes_train_fold1.json
✅ dataset_info['aes_train_fold1'] 已成功添加并保存到 /mnt/afs1/panhaojun/project/LLaMA-Factory/data/dataset_info.json
✅ message 已保存到: /mnt/afs1/panhaojun/project/LLaMA-Factory/data/aes_train_fold2.json
✅ dataset_info['aes_train_fold2'] 已成功添加并保存到 /mnt/afs1/panhaojun/project/LLaMA-Factory/data/dataset_info.json
✅ message 已保存到: /mnt/afs1/panhaojun/project/LLaMA-Factory/data/aes_train_fold3.json
✅ dataset_info['aes_train_fold3'] 已成功添加并保存到 /mnt/afs1/panhaojun/project/LLaMA-Factory/data/dataset_info.json
✅ message 已保存到: /mnt/afs1/panhaojun/project/LLaMA-Factory/data/aes_train_fold4.json
✅ dataset_info['aes_train_fold4'] 已成功添加并保存到 /mnt/afs1/panhaojun/project/LLaMA-Factory/data/dataset_info.json
✅ message 已保存到: /mnt/afs1/panhaojun